In [6]:
#softmax classifier
"""meet cross entropy cost function"""
import numpy as np
Y=np.array([1,0,0])

Y_pred1=np.array([.7,.2,.1])
Y_pred2=np.array([.1,.3,.6])

print("loss1 =",np.sum(-Y * np.log(Y_pred1)))
print("loss2 =",np.sum(-Y * np.log(Y_pred2)))

loss1 = 0.356674943939
loss2 = 2.30258509299


In [7]:
import torch
import torch.nn as nn
from torch.autograd import Variable
#softmax + crossEntropy (logsoftmax + NLLLoss)
loss=nn.CrossEntropyLoss()
#input is class , not one-hot
Y=Variable(torch.LongTensor([1]),requires_grad=False)

print("origin Y:",Y.data)
Y_pred1=Variable(torch.Tensor([[2,1,.1]]))
Y_pred2=Variable(torch.Tensor([[.5,2,.3]]))
print("Y_pred1 and Y_pred2 respectively:",Y_pred1.data[0],Y_pred2.data[0])

origin Y: 
 1
[torch.LongTensor of size 1]

Y_pred1 and Y_pred2 respectively: 
 2.0000
 1.0000
 0.1000
[torch.FloatTensor of size 3]
 
 0.5000
 2.0000
 0.3000
[torch.FloatTensor of size 3]



In [8]:
l1=loss(Y_pred1,Y)
l2=loss(Y_pred2,Y)

print("PyTorch Loss1 = ",l1.data[0],"\nPyTorch Loss2 = ",l2.data[0])

PyTorch Loss1 =  1.4170299768447876 
PyTorch Loss2 =  0.34061625599861145


In [9]:
#multi - class cross entropy
#Input is class, not one-hot
Y=Variable(torch.LongTensor([2,0,1]),requires_grad=False)

Y_pred1=Variable(torch.Tensor([
    [.1,.2,.9],
    [1.1,.1,.2],
    [.2,2.1,.1]
]))

Y_pred2=Variable(torch.Tensor([
    [.8,.2,.3],
    [.2,.3,.5],
    [.2,.2,.5]
]))

l1=loss(Y_pred1,Y)
l2=loss(Y_pred2,Y)

print("Batch loss1 = ",l1.data,
     "Batch loss2 = ",l2.data)

Batch loss1 =  
 0.4966
[torch.FloatTensor of size 1]
 Batch loss2 =  
 1.2389
[torch.FloatTensor of size 1]



In [10]:
#MNIST network
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)


model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299579
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.295074
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.304287
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.303878
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.303464
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.307876
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.304289
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.307246
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.296754
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.304306
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.306416
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.293647
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.303074
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.298885
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.301493
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.299237
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.296509
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.293153
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.296650
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.293911
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.536434
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.406437
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.620933
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.319608
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.588222
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.465288
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.449132
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.766084
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.620121
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.659462
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.540698
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.376042
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.523389
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.433859
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.512693
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.436751
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.432187
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.427430
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.338818
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.408107


Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.159066
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.327948
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.310430
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.159340
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.230155
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.378066
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.118264
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.173705
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.171536
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.240716
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.198605
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.228165
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.231055
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.281384
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.203165
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.138179
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.263377
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.112837
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.207075
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.225703


Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.143917
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.169265
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.123811
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.126487
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.221831
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.116496
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.335659
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.148660
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.087944
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.112202
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.137257
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.180279
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.122428
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.066357
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.064321
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.049621
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.214223
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.454920
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.131150
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.099825


Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.055410
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.164058

Test set: Average loss: 0.0020, Accuracy: 9632/10000 (96%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.029939
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.075931
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.072579
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.029892
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.187773
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.220245
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.078012
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.075554
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.097099
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.211875
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.348297
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.107932
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.034169
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.028810
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.184355
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.129414
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.069570
Train Epoch: 8 [10

Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.154207
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.072973
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.147997
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.168235
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.071916
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.121457
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.180527
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.092552
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.039240
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.177612
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.128714
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.070906
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.059725
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.103105
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.094869
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.111156
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.056421
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.095351
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.074052
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.066774
